In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import jieba
import fasttext
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
import os
import glob
from tqdm import tqdm

In [2]:
answer_train=pd.read_csv('answer_train.csv')

In [3]:
encode_dic={}
cate=answer_train.label.unique()
for i in range(len(cate)):
    encode_dic[cate[i]]=i
answer_train['label']=answer_train['label'].map(encode_dic)
filename_label={}
for index,filename,label in answer_train.itertuples():
    filename_label[filename]=label
#

In [4]:
#
decode_dic={}
for key in encode_dic.keys():
    decode_dic[encode_dic[key]]=key
#decode_dic

In [5]:
# train_data=pd.read_csv("dataset/train_val_df_127_100.csv")
# test_data=pd.read_csv("dataset/test2_df_127_100.csv")
# test_data_1=pd.read_csv("dataset/test1_df_127_100.csv")

In [6]:
train_data.shape,test_data.shape,test_data_1.shape

((59985, 2), (25439, 2), (8000, 2))

In [7]:
pseudo_csv=pd.read_csv('submit_a_976.csv')
pseudo_label=[encode_dic[i] for i in pseudo_csv['label'].values.tolist()]
test_data_1['label']=pseudo_label
test_data_1.drop(['name'],axis=1,inplace=True)
train_data_all=pd.concat((train_data,test_data_1)).reset_index(drop=True)

In [6]:
#train_data_all

In [9]:
def get_list_list(list_1,list_2):
    return [list_1[i] for i in list_2]

In [7]:
# #五折
# #直接利用文件名称即可训练一个fastText模型
# from sklearn.model_selection import StratifiedKFold

# #
# save_data='fastTextData_kfold/'
# if not os.path.exists(save_data):
#     os.mkdir(save_data)
# train_txt1=open(save_data+'train_1.txt','w',encoding='utf-8')
# val_txt1=open(save_data+'val_1.txt','w',encoding='utf-8')
# train_txt2=open(save_data+'train_2.txt','w',encoding='utf-8')
# val_txt2=open(save_data+'val_2.txt','w',encoding='utf-8')
# train_txt3=open(save_data+'train_3.txt','w',encoding='utf-8')
# val_txt3=open(save_data+'val_3.txt','w',encoding='utf-8')
# train_txt4=open(save_data+'train_4.txt','w',encoding='utf-8')
# val_txt4=open(save_data+'val_4.txt','w',encoding='utf-8')
# train_txt5=open(save_data+'train_5.txt','w',encoding='utf-8')
# val_txt5=open(save_data+'val_5.txt','w',encoding='utf-8')
# #
# index=0
# train_all = train_data_all['text'].values.tolist()
# kind = train_data_all['label'].values.tolist()
# n_splits = 5
# sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=20)
# fold = 0
# for train_idx, test_idx in sk.split(train_all, kind):
#     fold+=1
    
#     print(fold)
#     X_train = get_list_list(train_all, train_idx)
#     y_train = get_list_list(kind, train_idx)
#     X_test = get_list_list(train_all, test_idx)
#     y_test = get_list_list(kind, test_idx)
    
#     for tra_inx in range(len(X_train)):
#         text=X_train[tra_inx]
#         label=y_train[tra_inx]
#         seg_text = jieba.cut(text)
#         out_str = " ".join(seg_text)
#         if fold==1:
#             train_txt1.write('%s __label__%d\n' % (out_str, int(label)))
#         elif fold==2:
#             train_txt2.write('%s __label__%d\n' % (out_str, int(label)))
#         elif fold==3:
#             train_txt3.write('%s __label__%d\n' % (out_str, int(label)))
#         elif fold==4:
#             train_txt4.write('%s __label__%d\n' % (out_str, int(label)))
#         else:
#             train_txt5.write('%s __label__%d\n' % (out_str, int(label)))
#     for tes_inx in range(len(X_test)):
#         text=X_train[tes_inx]
#         label=y_train[tes_inx]
#         seg_text = jieba.cut(text)
#         out_str = " ".join(seg_text)
#         if fold==1:
#             val_txt1.write('%s __label__%d\n' % (out_str, int(label)))
#         elif fold==2:
#             val_txt2.write('%s __label__%d\n' % (out_str, int(label)))
#         elif fold==3:
#             val_txt3.write('%s __label__%d\n' % (out_str, int(label)))
#         elif fold==4:
#             val_txt4.write('%s __label__%d\n' % (out_str, int(label)))
#         else:
#             val_txt5.write('%s __label__%d\n' % (out_str, int(label)))
# #
# train_txt1.close()
# val_txt1.close()
# train_txt2.close()
# val_txt2.close()
# train_txt3.close()
# val_txt3.close()
# train_txt4.close()
# val_txt4.close()
# train_txt5.close()
# val_txt5.close()


In [ ]:
# 2.训练模型
import random
random.seed(2020)
model=fasttext.train_supervised('fastTextData_kfold/train_5.txt',lr=0.5, epoch=10, wordNgrams=2)
#print(model.words)
#print(model.labels)
#验证
texts_test, labels_test = [], []
with open('fastTextData_kfold/val_5.txt', 'r',encoding='utf-8') as f:
    for line in f:
        *text, label = line.strip().split(' ')
        text = ' '.join(text)
        texts_test.append(text)
        labels_test.append(label)
#   
label_encoder = preprocessing.LabelEncoder()
labels_test = label_encoder.fit_transform(labels_test)
predits = list(zip(*(model.predict(texts_test)[0])))[0]
predits = label_encoder.transform(predits)
score = metrics.accuracy_score(predits, labels_test)
print('accuracy_score : %.03f' % score)


In [ ]:
save_model='model_save_dir/'
if not os.path.exists(save_model):
    os.mkdir(save_model)
save_model=os.path.join(save_model,'fastext_5_'+str(int(score*1000))+'.bin')
model.save_model(save_model)
save_model
# 加载模型
#model = fasttext.load_model(save_model)

In [ ]:
#
sub_sample=pd.read_csv('submit_example_test2.csv')
sub_filenames=sub_sample['filename'].values
pres=[]
test_paths=glob.glob('test2/*')
test_paths=[sample.replace('\\','/') for sample in test_paths]
#
index=0
for _,name,text in tqdm(test_data.itertuples()):
    # 结巴分词
    seg_text = jieba.cut(text)
    out_str = " ".join(seg_text)
    #
    pres.append(int(model.predict(out_str)[0][0].split('_')[-1]))
#
#解码
pres=[decode_dic[i] for i in pres]
pres

In [6]:
model1=fasttext.load_model("model_save_dir/fastext_1_995.bin")
model2=fasttext.load_model("model_save_dir/fastext_2_995.bin")
model3=fasttext.load_model("model_save_dir/fastext_3_995.bin")
model4=fasttext.load_model("model_save_dir/fastext_4_996.bin")
model5=fasttext.load_model("model_save_dir/fastext_5_996.bin")

In [14]:
test_data

,name,text
0,test2/a7fd17bf2eeb85132a1390dcfd052624.xls,有限公司威海威海市环翠区刘伟光石忠东工程荣军建筑设计任力姜国旭陈玲娜姜夕武岩土双立姜夕车库郑...
1,test2/辽阳市辽阳县省执法证件信息_.csv,辽阳市辽阳县省执法证件信息
2,test2/邯郸市成安县常住人口.csv,邯郸市成安县常住人口
3,test2/e09b5a9fd3166849eab76945e036d3c5.xls,有限公司天津应用与及技术天津市的研究天津大学研究院中国关键技术集团科技股份石油工程中科学奖种...
4,test2/ffa49ad5e4d079e2e0b669ab9d76fc7b.xls,工业
...,...,...
25434,test2/云南省普洱市景东彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,云南省普洱市景东彝族自治县市汶上县残疾人联合会残疾人证办理
25435,test2/22fdcf3d324a752099cd77bff03d10a6.xls,有限公司天津应用与及技术天津市的研究天津大学研究院中国关键技术集团科技股份石油工程中科学奖种...
25436,test2/云南省昆明市晋宁县区法人行政许可数据.csv,云南省昆明市晋宁县区法人行政许可数据
25437,test2/大洼区区法人行政处罚数据.csv,大洼区区法人行政处罚数据


In [18]:
#
test_data=pd.read_csv("test2_df_127_100.csv")
sub_sample=pd.read_csv('submit_example_test2.csv')
sub_filenames=sub_sample['filename'].values
pres=[]
test_paths=glob.glob('test2/*')
test_paths=[sample.replace('\\','/') for sample in test_paths]
#
index=0
for _,name,text in tqdm(test_data.itertuples()):
    # 结巴分词
    seg_text = jieba.cut(text)
    out_str = " ".join(seg_text)
    #
    vote1=int(model1.predict(out_str)[0][0].split('_')[-1])
    vote2=int(model2.predict(out_str)[0][0].split('_')[-1])
    vote3=int(model3.predict(out_str)[0][0].split('_')[-1])
    vote4=int(model4.predict(out_str)[0][0].split('_')[-1])
    vote5=int(model5.predict(out_str)[0][0].split('_')[-1])
    tmp_list={}
    for i in range(20):
        tmp_list[i]=0
    #
    merges=[vote1,vote2,vote3,vote4,vote5]
    for k in merges:
        tmp_list[k]+=1
    #print(tmp)
    most=sorted(tmp_list.items(),key=lambda item:item[1])[-1][0]
    pres.append(most)
#
#解码
pres=[decode_dic[i] for i in pres]
pres

25439it [10:00, 42.35it/s]


['城乡建设',
 '政法监察',
 '经济管理',
 '教育科技',
 '工业',
 '生态环境',
 '医疗卫生',
 '文秘行政',
 '生态环境',
 '工业',
 '教育科技',
 '交通运输',
 '文化休闲',
 '城乡建设',
 '交通运输',
 '经济管理',
 '气象水文测绘地震地理',
 '交通运输',
 '经济管理',
 '财税金融',
 '生态环境',
 '城乡建设',
 '教育科技',
 '政法监察',
 '城乡建设',
 '政法监察',
 '工业',
 '工业',
 '教育科技',
 '工业',
 '文化休闲',
 '农业畜牧业',
 '交通运输',
 '文化休闲',
 '财税金融',
 '文化休闲',
 '文秘行政',
 '教育科技',
 '旅游服务',
 '农业畜牧业',
 '文化休闲',
 '医疗卫生',
 '医疗卫生',
 '生态环境',
 '财税金融',
 '工业',
 '城乡建设',
 '工业',
 '教育科技',
 '城乡建设',
 '教育科技',
 '教育科技',
 '工业',
 '医疗卫生',
 '交通运输',
 '生态环境',
 '教育科技',
 '文秘行政',
 '城乡建设',
 '劳动人事',
 '工业',
 '文化休闲',
 '文秘行政',
 '工业',
 '工业',
 '文化休闲',
 '医疗卫生',
 '生态环境',
 '医疗卫生',
 '城乡建设',
 '教育科技',
 '经济管理',
 '工业',
 '医疗卫生',
 '教育科技',
 '教育科技',
 '文秘行政',
 '工业',
 '文化休闲',
 '文化休闲',
 '农业畜牧业',
 '教育科技',
 '医疗卫生',
 '工业',
 '工业',
 '财税金融',
 '医疗卫生',
 '财税金融',
 '教育科技',
 '医疗卫生',
 '文化休闲',
 '财税金融',
 '文化休闲',
 '政法监察',
 '交通运输',
 '教育科技',
 '医疗卫生',
 '城乡建设',
 '经济管理',
 '文化休闲',
 '财税金融',
 '旅游服务',
 '文化休闲',
 '城乡建设',
 '文秘行政',
 '文秘行政',
 '教育科技',
 '政法监察',
 '农业畜牧业',
 '交通运输',
 '财税金融',
 '农业畜牧业',
 '工业',
 '教育

In [13]:
merges

[5, 5, 5, 5, 5]

In [19]:
sub_sample['label']=pres
sub_sample.to_csv('submit_12_7_v3.csv',index=False)
sub_sample

,filename,label
0,test2/a7fd17bf2eeb85132a1390dcfd052624.xls,城乡建设
1,test2/辽阳市辽阳县省执法证件信息_.csv,政法监察
2,test2/邯郸市成安县常住人口.csv,经济管理
3,test2/e09b5a9fd3166849eab76945e036d3c5.xls,教育科技
4,test2/ffa49ad5e4d079e2e0b669ab9d76fc7b.xls,工业
...,...,...
25434,test2/云南省普洱市景东彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生
25435,test2/22fdcf3d324a752099cd77bff03d10a6.xls,教育科技
25436,test2/云南省昆明市晋宁县区法人行政许可数据.csv,文秘行政
25437,test2/大洼区区法人行政处罚数据.csv,文秘行政


In [20]:
sub9007=pd.read_csv("9007.csv")

In [21]:
metrics.accuracy_score(sub9007['label'],sub_sample['label'])

0.9301466252604269

In [27]:
#纯标题的用于替换
only_title_df=pd.read_csv("only_title.csv")
title_cls=only_title_df['label'].values
title_name=only_title_df['filename'].values
dic_title={}
for i in range(len(title_cls)):
    dic_title[title_name[i]]=title_cls[i]
#
total_df=sub_sample['filename'].values
result=sub_sample['label'].values
for j in range(len(total_df)):
    if total_df[j] in dic_title:
        result[j]=dic_title[total_df[j]]#用纯标题的结果进行替换
#
submit = {'filename': sub_sample['filename'].values, 'label': result}
sub = pd.DataFrame(submit)
sub.to_csv("submit_12_7_v1_replace_title.csv",index=False)
sub

,filename,label
0,test2/a7fd17bf2eeb85132a1390dcfd052624.xls,城乡建设
1,test2/辽阳市辽阳县省执法证件信息_.csv,政法监察
2,test2/邯郸市成安县常住人口.csv,经济管理
3,test2/e09b5a9fd3166849eab76945e036d3c5.xls,教育科技
4,test2/ffa49ad5e4d079e2e0b669ab9d76fc7b.xls,工业
...,...,...
25434,test2/云南省普洱市景东彝族自治县市汶上县残疾人联合会_残疾人证办理_.csv,医疗卫生
25435,test2/22fdcf3d324a752099cd77bff03d10a6.xls,教育科技
25436,test2/云南省昆明市晋宁县区法人行政许可数据.csv,文秘行政
25437,test2/大洼区区法人行政处罚数据.csv,文秘行政


In [29]:
metrics.accuracy_score(sub_sample['label'],sub['label'])

1.0

In [30]:
metrics.accuracy_score(sub8137['label'],sub_sample['label'])

0.8237352097173631

In [31]:
metrics.accuracy_score(sub8137['label'],sub['label'])

0.8237352097173631